In [1]:
import sys
sys.path.insert(1, '../..')

import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv("../../data/processed/phemernr2_dataset_with_tvt.csv", lineterminator="\n")
data.head()

,tweet_id,tweet_text,label,label2,topic,tvt,cv_fold,tt,tvt2
0,552833795142209536,The East London Mosque would like to offer its...,non-rumours,non-rumours,charliehebdo-all-rnr-threads,test,2,test,training
1,580318210609696769,BREAKING - A Germanwings Airbus A320 plane rep...,rumours,true,germanwings-crash-all-rnr-threads,training,3,training,training
2,552798891994009601,Reports that two of the dead in the #CharlieHe...,rumours,true,charliehebdo-all-rnr-threads,test,2,test,validation
3,576790814942236672,After #Putin disappeared Russian TV no longer ...,non-rumours,non-rumours,putinmissing-all-rnr-threads,test,2,test,training
4,499678822598340608,Saw #Ferguson for myself. #justiceformichaelbr...,non-rumours,non-rumours,ferguson-all-rnr-threads,training,3,training,training


In [3]:
bigram_data = pd.read_excel('../../data/processed/twitter15_ngram_distribution.xlsx', sheet_name='bigram')
bigram_data.columns = ["token", "unverified", "non-rumor", "true", "false"]
bigram_data.head()

,token,unverified,non-rumor,true,false
0,ca kkk,1,0,0,0
1,kkk grand,1,0,0,0
2,grand wizard,1,0,0,0
3,wizard endorses,1,0,0,0
4,endorses @hillaryclinton,1,0,0,0


In [4]:
bigram_vector_base = bigram_data['token'].tolist()
print(len(bigram_vector_base))
bigram_vector_base[10]

13842


'to trump'

In [5]:
label_rnr = []
for i, d in data.iterrows():
    if d['label'] == "non-rumor":
        label_rnr.append("non-rumours")
    else:
        label_rnr.append("rumours")
        
data['label_rnr'] = pd.Series(label_rnr)
data.head()

,tweet_id,tweet_text,label,label2,topic,tvt,cv_fold,tt,tvt2,label_rnr
0,552833795142209536,The East London Mosque would like to offer its...,non-rumours,non-rumours,charliehebdo-all-rnr-threads,test,2,test,training,rumours
1,580318210609696769,BREAKING - A Germanwings Airbus A320 plane rep...,rumours,true,germanwings-crash-all-rnr-threads,training,3,training,training,rumours
2,552798891994009601,Reports that two of the dead in the #CharlieHe...,rumours,true,charliehebdo-all-rnr-threads,test,2,test,validation,rumours
3,576790814942236672,After #Putin disappeared Russian TV no longer ...,non-rumours,non-rumours,putinmissing-all-rnr-threads,test,2,test,training,rumours
4,499678822598340608,Saw #Ferguson for myself. #justiceformichaelbr...,non-rumours,non-rumours,ferguson-all-rnr-threads,training,3,training,training,rumours


In [6]:
# labels_str = ['rumour', 'non-rumour']
label_type = "label2"
labels_str = data[label_type].unique().tolist()
labels_str

['non-rumours', 'true', 'unverified', 'false']

In [7]:
labels = []
for i, d in data.iterrows():
    lab = labels_str.index(d[label_type])
    labels.append(lab)
labels[:10]

[0, 1, 1, 0, 0, 0, 0, 2, 0, 0]

In [8]:
import string
import nltk
from nltk.tokenize import TweetTokenizer

tokenizer = TweetTokenizer(reduce_len=True)


def text2unigrams(text):
    texts = tokenizer.tokenize(text.encode('ascii', 'ignore').decode('utf8'))
    texts = [t for t in texts if t not in string.punctuation]
    texts = [t for t in texts if t not in ['URL', '‘', '’']]
    
    unigrams = texts
    
    return unigrams


def text2bigrams(text):
    texts = tokenizer.tokenize(text.encode('ascii', 'ignore').decode('utf8'))
    texts = [t for t in texts if t not in string.punctuation]
    texts = [t for t in texts if t not in ['URL', '‘', '’']]
    
    bigrams = nltk.bigrams(texts)
    bigrams = map(' '.join, bigrams)
    bigrams = [bgr for bgr in bigrams]
    
    return bigrams


def text2trigrams(text):
    texts = tokenizer.tokenize(text.encode('ascii', 'ignore').decode('utf8'))
    texts = [t for t in texts if t not in string.punctuation]
    texts = [t for t in texts if t not in ['URL', '‘', '’']]
    
    trigrams = nltk.trigrams(texts)
    trigrams = map(' '.join, trigrams)
    trigrams = [bgr for bgr in trigrams]
    
    return trigrams


def bigrams_vectors_generation(texts):
    bigram_vectors = []
    for text in texts:
        bigrams = text2bigrams(text)

        init_vec = [0.0 for _ in range(len(bigram_vector_base) + 1)]
        for bgr in bigrams:
            if bgr in bigram_vector_base:
                idx = bigram_vector_base.index(bgr)
                init_vec[idx] = 1.0
            else:
                init_vec[-1] = 1.0
        bigram_vectors.append(init_vec)
    
    return bigram_vectors


def custom_vectors_generation(texts, vector_terms):
    vectors = []
    for text in texts:
        bigrams = text2bigrams(text)
        trigrams = text2trigrams(text)

        init_vec = [0.0 for _ in range(len(vector_terms) + 1)]
        for bgr in bigrams:
            if bgr in vector_terms:
                idx = vector_terms.index(bgr)
                init_vec[idx] = 1.0
            else:
                init_vec[-1] = 1.0
        for tgr in trigrams:
            if tgr in vector_terms:
                idx = vector_terms.index(tgr)
                init_vec[idx] = 1.0
            else:
                init_vec[-1] = 1.0
        vectors.append(init_vec)
    
    return vectors

In [9]:
texts = data['tweet_text'].tolist()
# vectors = bigrams_vectors_generation(texts)

In [10]:
import nltk
from nltk.collocations import *

top_n = 2000
bigram_measures = nltk.collocations.BigramAssocMeasures()
trigram_measures = nltk.collocations.TrigramAssocMeasures()

finder2 = BigramCollocationFinder.from_words([])
finder3 = TrigramCollocationFinder.from_words([])

# generating bigram and trigram
for text in texts:
    unigrams = text2unigrams(text)
    bigrams = text2bigrams(text)
    trigrams = text2trigrams(text)
    
    for ngrm in unigrams:
        if ngrm not in finder2.word_fd:
            finder2.word_fd[ngrm] = 0
        finder2.word_fd[ngrm] += 1
        finder2.N += 1
        
        if ngrm not in finder3.word_fd:
            finder3.word_fd[ngrm] = 0
        finder3.word_fd[ngrm] += 1
        finder3.N += 1
    
    for ngrm in bigrams:
        term = tuple([i for i in ngrm.split()])
        
        if term not in finder2.ngram_fd:
            finder2.ngram_fd[term] = 0
            
        finder2.ngram_fd[term] += 1

    for ngrm in trigrams:
        term = tuple([i for i in ngrm.split()])
        
        if term not in finder3.ngram_fd:
            finder3.ngram_fd[term] = 0
            
        finder3.ngram_fd[term] += 1
        
# only bigrams that appear 3+ times
finder2.apply_freq_filter(3)
finder3.apply_freq_filter(3)

combined = []
for res in finder2.score_ngrams(bigram_measures.pmi):
    combined.append(res)
for res in finder3.score_ngrams(trigram_measures.pmi):
    combined.append(res)
combined = sorted(combined, key=lambda x: x[1], reverse=True)

In [11]:
print(len(finder2.ngram_fd), len(finder3.ngram_fd))
count = 0
for k, v in combined:
    print(f"{k} - {v}")
    count += 1
    if count >= 10:
        break

5500 2977
('AG', 'Eric', 'Holder') - 30.1391512738226
('33', '55', '77') - 29.309076275264914
('Hormone', 'MV', 'Concept') - 29.309076275264914
('Last-minute', 'lawsuit', 'filed') - 27.5017213532073
('AC', 'Milan', 'midfielder') - 27.309076275264914
('serious', 'depressive', 'episode') - 26.899685339127203
('http://t.co/j1ljDbazOw', 'UPDATES', 'http://t.co/PD7pVfU9pQ') - 26.817223178935233
('Jewish', 'pro-censorship', 'lobby') - 26.665220085490187
('Mean', 'Russias', 'Undergoing') - 26.64729817749292
('altitude', 'speed', 'chart') - 26.50172135320731


In [12]:
term_vector_base = [" ".join(c[0]) for c in combined[:top_n]]
vectors = custom_vectors_generation(texts, term_vector_base)

In [13]:
len(vectors[102])

2001

In [14]:
min_score = 100
max_score = 0
n_2gram = 0
n_3gram = 0
for k, v in combined[:top_n]:
    min_score = min(min_score, v)
    max_score = max(max_score, v)
    
    if len(k) == 2:
        n_2gram += 1
    
    if len(k) == 3:
        n_3gram += 1
        
print(min_score)
print(max_score)
print(n_2gram)
print(n_3gram)

11.83010970221591
30.1391512738226
200
1800


In [15]:
train_vectors = np.array([vectors[i] for i, d in data.iterrows() if d['tvt2'] == 'training'])
val_vectors = np.array([vectors[i] for i, d in data.iterrows() if d['tvt2'] == 'validation'])
test_vectors = np.array([vectors[i] for i, d in data.iterrows() if d['tvt2'] == 'testting'])

train_labels = np.array([labels[i] for i, d in data.iterrows() if d['tvt2'] == 'training'])
val_labels = np.array([labels[i] for i, d in data.iterrows() if d['tvt2'] == 'validation'])
test_labels = np.array([labels[i] for i, d in data.iterrows() if d['tvt2'] == 'testting'])

In [16]:
from sklearn.svm import LinearSVC

from library.classification import SKLearnClassification
from library.evaluation import ConfusionMatrix

svm = LinearSVC()

model = SKLearnClassification(svm, "Support Vector Machine")
print(f"\n--- START ---")
model.train(train_vectors, train_labels, "Phemernr2")

print("Validation Set")
preds = model.predict(val_vectors)

conf_mat = ConfusionMatrix(
    labels=val_labels,
    predictions=preds,
    binary=True
)
conf_mat.evaluate()

print("--- END ---\n")


--- START ---
---> execution time : 0.13 seconds
Validation Set
Binary Class Evaluation

True Positive : 94
False Positive : 35
False Negative : 445
True Negative : 876

Class positive Evaluation
- Precision : 72.868 %
- Recall : 17.44 %
- F1 : 0.28144

Class negative Evaluation
- Precision : 66.313 %
- Recall : 96.158 %
- F1 : 0.78495

Combined Evaluation
- Accuracy : 66.897 %
- Precision : 69.591 %
- Recall : 56.799 %
- F1 : 0.62548
- Average Confidence : 134.07 %
Model, Combined,,,,positive,,,negative,,,
Anonymous, 66.897, 69.591, 56.799, 0.62548, 72.868, 17.44, 0.28144, 66.313, 96.158, 0.78495, 
--- END ---



In [17]:
from scipy.sparse import csr_matrix

coefs = model.model.coef_
if type(coefs) == csr_matrix:
    coefs.toarray().tolist()[0]
else:
    coefs.tolist()
coefs_and_features = list(zip(coefs[0], term_vector_base))

# Most predictive overall
# coefs_and_features = sorted(coefs_and_features, key=lambda x: x[0], reverse=True)
coefs_and_features = sorted(coefs_and_features, key=lambda x: abs(x[0]), reverse=True)
print(f"Total tokens : {len(coefs_and_features)}")

Total tokens : 2000


In [18]:
coefs.shape

(4, 2001)

In [19]:
coefs_and_features

[(1.6087695280165981, '12 people have'),
 (-1.5849987928517644, 'Chocolate Cafe in'),
 (-1.4951413037154313, 'release the name'),
 (-1.4603899462161742, 'die as martyrs'),
 (-1.3650129701758065, 'as heavily armed'),
 (1.363991912582164, 'explosions heard at'),
 (-1.359516778655409, 'Andreas Lubitz'),
 (-1.3390156316115893, 'Cpl Nathan Cirillo'),
 (-1.3338868139022104, 'at Massey Hall'),
 (-1.3211535427622028, 'gunmen were involved'),
 (-1.3178897608762807, 'Canadian soldier killed'),
 (-1.31678332568287, "isn't by Banksy"),
 (-1.2771673317763774, 'has been evacuated'),
 (-1.2704822313185646, 'were on board'),
 (-1.2688789570968737, 'University of Ottawa'),
 (-1.2576364578522676, 'show in Toronto'),
 (1.2507555752287112, 'If Michael Brown'),
 (-1.2447946883005776, 'Nathan Cirillo 24'),
 (-1.228784033439546, 'kosher market in'),
 (-1.2235617991632999, 'gunman shot dead'),
 (-1.175578995199724, 'holed up in'),
 (-1.1755789922878501, 'as Michael Zehaf-Bibeau'),
 (-1.1649609202071134, 'an u

In [20]:
# print(max(f1s))
# f1s.index(max(f1s))

In [21]:
# import matplotlib.pyplot as plt

# x = [i*100 for i in range(len(f1s))]
# y = f1s

# plt.xlabel("Token/Bigram used")
# plt.ylabel("F1 Score")
# plt.plot(x, y)
# plt.show()

In [22]:
# import json

# best_tokens = [t[1] for t in coefs_and_features[:6100]]
# best_tokens[:10]
# 
# with open("../../data/processed/twitter15_best_bigrams.txt", "w") as f:
#     for token in best_tokens:
#         f.write(token + "\n")

In [23]:
coefs_and_features[:10]

[(1.6087695280165981, '12 people have'),
 (-1.5849987928517644, 'Chocolate Cafe in'),
 (-1.4951413037154313, 'release the name'),
 (-1.4603899462161742, 'die as martyrs'),
 (-1.3650129701758065, 'as heavily armed'),
 (1.363991912582164, 'explosions heard at'),
 (-1.359516778655409, 'Andreas Lubitz'),
 (-1.3390156316115893, 'Cpl Nathan Cirillo'),
 (-1.3338868139022104, 'at Massey Hall'),
 (-1.3211535427622028, 'gunmen were involved')]

In [24]:
coefs_and_features[-10:]

[(0.0, 'responsible in their'),
 (0.0, 'Centre and Parliament'),
 (0.0, 'Kabaeva giving'),
 (0.0, 'RCMP advises'),
 (0.0, 'down the street'),
 (0.0, "don't need to"),
 (0.0, 'due to his'),
 (0.0, 'at least 2'),
 (0.0, 'friends of those'),
 (0.0, 'altitude speed')]

In [25]:
coefs_and_features[-10:]

[(0.0, 'responsible in their'),
 (0.0, 'Centre and Parliament'),
 (0.0, 'Kabaeva giving'),
 (0.0, 'RCMP advises'),
 (0.0, 'down the street'),
 (0.0, "don't need to"),
 (0.0, 'due to his'),
 (0.0, 'at least 2'),
 (0.0, 'friends of those'),
 (0.0, 'altitude speed')]

In [26]:
n_best = 750
best_tokens = []

# for cf in coefs_and_features[-n_best:]:
#     best_tokens.append(cf[1])

for cf in coefs_and_features[:n_best]:
    best_tokens.append(cf[1])
    
print(best_tokens[:10])
    
with open("../../data/processed/phemernr2_best_terms.txt", "w") as f:
    for token in best_tokens:
        f.write(token + "\n")

['12 people have', 'Chocolate Cafe in', 'release the name', 'die as martyrs', 'as heavily armed', 'explosions heard at', 'Andreas Lubitz', 'Cpl Nathan Cirillo', 'at Massey Hall', 'gunmen were involved']
